# Redes Neurais Convolutionais

Na seção anterior, construímos e treinamos um modelo simples para classificar imagens ASL. O modelo foi capaz de aprender como classificar corretamente o conjunto de dados de treinamento com precisão muito alta, mas não teve um desempenho tão bom no conjunto de dados de validação. Esse comportamento de não generalizar bem para dados que não são de treinamento é chamado de [overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html) e nesta seção apresentaremos um tipo popular de modelo chamado [rede neural convolucional](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) que é especialmente bom para ler imagens e classificá-las.

## Tarefas

* Prepare dados especificamente para uma CNN
* Crie um modelo CNN mais sofisticado, compreendendo uma maior variedade de camadas do modelo
* Treine um modelo CNN e observe seu desempenho

## Carregando e preparando os dados

A célula abaixo contém as técnicas de pré-processamento de dados, Execute-o antes de prosseguir:

In [ ]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("data/mnist/sign_mnist_train.csv")
valid_df = pd.read_csv("data/mnist/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## Remodelando (Reshaping)  das Imagens para uma CNN

As imagens individuais do nosso conjunto de dados estão no formato de longas listas de 784 pixels:



In [ ]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

Neste formato, não temos todas as informações sobre quais pixels estão próximos uns dos outros. Por causa disso, não podemos aplicar convoluções que detectem recursos. Vamos remodelar nosso conjunto de dados para que fiquem no formato de 28x28 pixels. Isso permitirá que nossas convoluções associem grupos de pixels e detectem recursos importantes.

Observe que para a primeira camada convolucional do nosso modelo, precisamos ter não apenas a altura e a largura da imagem, mas também o número de [canais de cores](https://www.photoshopessentials.com/essentials/rgb/) . Nossas imagens estão em tons de cinza, então teremos apenas 1 canal.

Isso significa que precisamos converter a forma atual `(27455, 784)` para `(27455, 28, 28, 1)`. Por conveniência, podemos passar ao método [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) um `-1` para qualquer dimensão que desejarmos permanecem os mesmos, portanto:

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [ ]:
x_train.shape

(27455, 28, 28, 1)

In [ ]:
x_valid.shape

(7172, 28, 28, 1)

In [ ]:
x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

## Criando um Modelo Convolutional

Hoje em dia, muitos cientistas de dados iniciam seus projetos pegando emprestadas propriedades de modelo de um projeto semelhante. Supondo que o problema não seja totalmente único, há uma grande chance de que as pessoas tenham criado modelos com bom desempenho e que sejam publicados em repositórios on-line como o [TensorFlow Hub](https://www.tensorflow.org/hub) e o [Catálogo NGC ](https://ngc.nvidia.com/catalog/models). Hoje, forneceremos um modelo que funcionará bem para esse problema.

<img src="https://drive.google.com/uc?id=1WeMvcGJiRLmXCsp6kwozb-7slhHboooq" width=180 />

Abordamos muitos dos diferentes tipos de camadas na palestra e abordaremos todas elas aqui com links para sua documentação. Em caso de dúvida, leia a documentação oficial (ou pergunte ao [stackoverflow](https://stackoverflow.com/)).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", input_shape=(28, 28, 1))) # camada convulocional que soma os pixel
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))

model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))

model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization()) #  normalização em lote dimensiona os valores nas camadas ocultas para melhorar o treinamento
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten()) # tansforma a matriz em um vetor

model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3)) # desliga 30% dos neuronios de forma aleatoria
model.add(Dense(units=num_classes, activation="softmax"))

### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<img src="https://drive.google.com/uc?id=1f4iuQX4IBOKW1bCAs9O_MTP-bqBlj-aQ" largura=300 />

Estas são nossas camadas convolucionais 2D. Kernels pequenos examinarão a imagem de entrada e detectarão recursos que são importantes para a classificação. As convoluções anteriores do modelo detectarão recursos simples, como linhas. As convoluções posteriores detectarão recursos mais complexos. Vejamos nossa primeira camada Conv2D:
```Python
model.add(Conv2D(75 , (3,3) , passadas = 1 , preenchimento = 'mesmo'...)
```
75 refere-se ao número de filtros que serão aprendidos. (3,3) refere-se ao tamanho desses filtros. Os avanços referem-se ao tamanho do passo que o filtro executará ao passar pela imagem. O preenchimento refere-se a se a imagem de saída criada a partir do filtro corresponderá ao tamanho da imagem de entrada.

### [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) (Normalização em Lotes)

Assim como a normalização de nossas entradas, a normalização em lote dimensiona os valores nas camadas ocultas para melhorar o treinamento. [Leia mais sobre isso em detalhes aqui](https://blog.paperspace.com/busting-the-myths-about-batch-normalization/).

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

<img src="https://drive.google.com/uc?id=1V0KkSlK9mSrJJLSt6dZM-dN1ciUJJYH2" width=300 />

O Max pooling pega uma imagem e essencialmente a reduz para uma resolução mais baixa. Ele faz isso para ajudar o modelo a ser robusto à translação (objetos movendo-se de um lado para o outro) e também torna nosso modelo mais rápido.

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

<img src="https://drive.google.com/uc?id=1Xd4HFauuDicUANuPu-sgTuUOjJ_Rhnat" width=360 />

Dropout é uma técnica para prevenir overfitting. O dropout seleciona aleatoriamente um subconjunto de neurônios e os desliga, para que eles não participem da propagação direta ou reversa naquela passagem específica. Isso ajuda a garantir que a rede seja robusta e redundante e não dependa de nenhuma área para encontrar respostas.

### [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)

Flatten pega a saída de uma camada que é multidimensional e a nivela em uma matriz unidimensional. A saída é chamada de vetor de características e será conectada à camada de classificação final.

### [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

Já vimos camadas densas em nossos modelos anteriores. Nossa primeira camada densa (512 unidades) toma o vetor de recursos como entrada e aprende quais recursos contribuirão para uma classificação específica. A segunda camada densa (24 unidades) é a camada de classificação final que produz nossa previsão.


## Resumo do Modelo

Isso pode parecer muita informação, mas não se preocupe. Não é fundamental entender tudo agora para treinar modelos convolucionais com eficácia. Mais importante ainda, sabemos que eles podem ajudar na extração de informações úteis de imagens e podem ser usados em tarefas de classificação.

Aqui, resumimos o modelo que acabamos de criar. Observe como ele tem menos parâmetros treináveis que o modelo do notebook anterior:

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 75)        750       
                                                                 
 batch_normalization_3 (Bat  (None, 28, 28, 75)        300       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 75)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout_2 (Dropout)         (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_4 (Bat  (None, 14, 14, 50)       

## Compilando o Modelo

Compilaremos o modelo como antes:

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Treinando o Modelo

Apesar da arquitetura do modelo muito diferente, o treinamento parece exatamente o mesmo. Execute a célula abaixo para treinar por 20 épocas e vamos ver se a precisão melhora:

In [ ]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/20
858/858 [==============================] - 20s 10ms/step - loss: 0.3091 - accuracy: 0.9058 - val_loss: 0.4714 - val_accuracy: 0.8652
Epoch 2/20
858/858 [==============================] - 7s 8ms/step - loss: 0.0192 - accuracy: 0.9938 - val_loss: 0.3927 - val_accuracy: 0.8900
Epoch 3/20
858/858 [==============================] - 7s 8ms/step - loss: 0.0108 - accuracy: 0.9966 - val_loss: 0.1687 - val_accuracy: 0.9625
Epoch 4/20
858/858 [==============================] - 7s 8ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 0.4116 - val_accuracy: 0.9098
Epoch 5/20
858/858 [==============================] - 8s 9ms/step - loss: 0.0032 - accuracy: 0.9992 - val_loss: 0.2503 - val_accuracy: 0.9416
Epoch 6/20
858/858 [==============================] - 7s 8ms/step - loss: 0.0034 - accuracy: 0.9990 - val_loss: 0.1768 - val_accuracy: 0.9484
Epoch 7/20
858/858 [==============================] - 7s 8ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.1541 - val_accuracy: 0.9603
Epoc

## Discussões dos resultados

Parece que este modelo foi significativamente melhorado! A precisão do treinamento é muito alta e a precisão da validação também melhorou. Este é um ótimo resultado, pois tudo o que tivemos que fazer foi trocar por um novo modelo.

Você deve ter notado a precisão da validação variando. Isto é uma indicação de que nosso modelo ainda não está generalizando perfeitamente. Felizmente, há mais que podemos fazer. Vamos falar sobre isso na próxima palestra.

## Summary

Nesta seção, utilizamos vários novos tipos de camadas para implementar uma CNN, que teve um desempenho melhor do que o modelo mais simples usado na última seção. Esperamos que o processo geral de criação e treinamento de um modelo com dados preparados esteja começando a se tornar ainda mais familiar.

## Limpando a memória
Antes de prosseguir, execute a seguinte célula para limpar a memória da GPU. Isso é necessário para passar para o próximo caderno.

In [ ]:
#import IPython
#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

## Próximo

Nas últimas seções você se concentrou na criação e no treinamento de modelos. Para melhorar ainda mais o desempenho, agora você voltará sua atenção para o *aumento de dados*, uma coleção de técnicas que permitirá que seus modelos sejam treinados com mais e melhores dados do que aqueles que você poderia ter originalmente à sua disposição.